# Phase 1 - Data & Descriptive Statistics

## Data Ingestion & Initial Inspection

This notebook begins the analysis of NYC yellow taxi ride demand (Jan-Mar 2019).  

**Objectives of this notebook:**
- Load and concatenate the raw CSV data
- Inspect schema, datatypes, and missing values
- Document initial observations and potential data quality issues


In [25]:
import sys
import os
import pandas as pd
import numpy as np
from   importlib import reload

# Add src directory to path
sys.path.append("../../src")

# Import the load_data module
from data import load_data


In [26]:
# Load all CSVs and concatenate into a single DataFrame
df = load_data.load_raw_taxi_data()

Loading ../../data/raw/yellow_tripdata_2019-01.csv...
Loading ../../data/raw/yellow_tripdata_2019-02.csv...
Loading ../../data/raw/yellow_tripdata_2019-03.csv...


In [30]:
# Inspect the concatenated dataframe
load_data.inspect_schema(df)


--- Schema Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22519712 entries, 0 to 22519711
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        int64         
 4   trip_distance          float64       
 5   RatecodeID             int64         
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18 

## Initial Observations

- Dataset contains ~22.5 million rows and 19 columns.
- Column names and types match TLC documentation, though casing differs from the project data dictionary.
- Some pickup datetimes fall outside the intended Jan-Mar 2019 window.
- Congestion surcharge is missing for ~22% of records; this is structural (introduced in early 2019 and only applied to certain zones/times).
- Some trips have unrealistic fare-duration combinations and will require cleaning.
- Further cleaning and feature engineering will be performed later on.

In [32]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,source_file
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN,yellow_tripdata_2019-01.csv
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN,yellow_tripdata_2019-01.csv
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN,yellow_tripdata_2019-01.csv
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN,yellow_tripdata_2019-01.csv
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN,yellow_tripdata_2019-01.csv


In [31]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,2.251971e+07,22519712,22519712,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,2.251971e+07,1.766373e+07
mean,1.636566e+00,2019-02-15 07:40:05.223035904,2019-02-15 07:57:10.702980608,1.571249e+00,2.893291e+00,1.062180e+00,1.636597e+02,1.619358e+02,1.280282e+00,1.278228e+01,8.911029e-01,4.961151e-01,2.072678e+00,3.413585e-01,2.991298e-01,1.779865e+01,1.875677e+00
min,1.000000e+00,2001-02-02 14:55:07,2001-02-02 15:07:27,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-4.470000e+02,-6.000000e+01,-5.000000e-01,-8.989000e+01,-7.000000e+01,-3.000000e-01,-4.503000e+02,-2.500000e+00
25%,1.000000e+00,2019-01-24 11:05:38,2019-01-24 11:24:48,1.000000e+00,9.400000e-01,1.000000e+00,1.140000e+02,1.120000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,9.960000e+00,2.500000e+00
50%,2.000000e+00,2019-02-14 21:57:50,2019-02-14 22:14:17,1.000000e+00,1.600000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,1.700000e+00,0.000000e+00,3.000000e-01,1.350000e+01,2.500000e+00
75%,2.000000e+00,2019-03-09 13:28:09,2019-03-09 13:45:17.249999872,2.000000e+00,2.920000e+00,1.000000e+00,2.330000e+02,2.340000e+02,2.000000e+00,1.400000e+01,1.000000e+00,5.000000e-01,2.750000e+00,0.000000e+00,3.000000e-01,1.900000e+01,2.500000e+00
max,4.000000e+00,2088-01-24 00:25:39,2088-01-24 07:28:25,9.000000e+00,8.318000e+02,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,9.432748e+05,5.353800e+02,7.500000e+01,1.414920e+05,3.288000e+03,1.000000e+00,1.084772e+06,4.500000e+00
std,5.260722e-01,NaN,NaN,1.226990e+00,3.805281e+00,6.729183e-01,6.608905e+01,7.024575e+01,4.706301e-01,2.926392e+02,1.152017e+00,5.903604e-02,2.993208e+01,1.801560e+00,2.208043e-02,3.137936e+02,1.086081e+00


In [36]:
# Save concatencated raw data
df.to_parquet("../../data/processed/raw_combined_2019_q1.parquet", index=False, engine="fastparquet")